In [1]:
import cv2
import dlib

def test_face_detection(image_path):
    detector = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    faces = detector.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    if len(faces) > 0:
        return True
    else:
        return False

        
def test_face_detection_dlib(image_path):
    detector = dlib.get_frontal_face_detector()
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    faces = detector(gray)
    
    if len(faces) > 0:
        return True
    else:
        return False

    

# Example Usage
for i in range(1, 18):
    image_path = f"testing_dataset/{i:02d}.jpg"
    x = test_face_detection(image_path)
    y = test_face_detection_dlib(image_path)
    if x or y:
        print(f"Face detected in {image_path}")
    else:
        print(f"No face detected in {image_path}")



Face detected in testing_dataset/01.jpg
Face detected in testing_dataset/02.jpg
Face detected in testing_dataset/03.jpg
Face detected in testing_dataset/04.jpg
Face detected in testing_dataset/05.jpg
Face detected in testing_dataset/06.jpg
Face detected in testing_dataset/07.jpg
Face detected in testing_dataset/08.jpg
Face detected in testing_dataset/09.jpg
Face detected in testing_dataset/10.jpg
Face detected in testing_dataset/11.jpg
Face detected in testing_dataset/12.jpg
Face detected in testing_dataset/13.jpg
Face detected in testing_dataset/14.jpg
Face detected in testing_dataset/15.jpg
Face detected in testing_dataset/16.jpg
Face detected in testing_dataset/17.jpg


In [2]:
import cv2
import dlib
from imutils import face_utils
import os

def test_face_and_landmark_detection(image_path, predictor):
    haar_detector = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    dlib_detector = dlib.get_frontal_face_detector()
    
    image = cv2.imread(image_path)
    if image is None:
        print(f"Image file {image_path} cannot be loaded.")
        return
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    haar_faces = haar_detector.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    haar_face_detected = len(haar_faces) > 0
    
    dlib_faces = dlib_detector(gray)
    dlib_face_detected = len(dlib_faces) > 0

    if haar_face_detected or dlib_face_detected:
        print(f"Face detected in {image_path} ")
        
        if haar_face_detected:
            for (x, y, w, h) in haar_faces:
                rect = dlib.rectangle(int(x), int(y), int(x + w), int(y + h))
                shape = predictor(gray, rect)
                shape = face_utils.shape_to_np(shape)
                
                if len(shape) == 68:
                    print(f"Facial landmarks detected in {image_path}")
                    for (lx, ly) in shape:
                        cv2.circle(image, (lx, ly), 2, (0, 255, 0), -1)
                else:
                    print(f"Failed to detect all landmarks in {image_path}")
        elif dlib_face_detected:
            for rect in dlib_faces:
                shape = predictor(gray, rect)
                shape = face_utils.shape_to_np(shape)
                
                if len(shape) == 68:
                    print(f"Facial landmarks detected in {image_path}")
                    for (lx, ly) in shape:
                        cv2.circle(image, (lx, ly), 2, (0, 255, 0), -1)
                else:
                    print(f"Failed to detect all landmarks in {image_path}")
        
        #Uncomment the following lines if you need to visualize the results
        # cv2.imshow("Landmarks", image)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()
    else:
        print(f"No face detected in {image_path}")

predictor_path = 'shape_predictor_68_face_landmarks.dat'
predictor = dlib.shape_predictor(predictor_path)

for i in range(1, 18):
    image_path = f"testing_dataset/{i:02d}.jpg"
    if not os.path.exists(image_path):
        print(f"Image file {image_path} does not exist.")
        continue

    test_face_and_landmark_detection(image_path, predictor)


Face detected in testing_dataset/01.jpg 
Facial landmarks detected in testing_dataset/01.jpg
Face detected in testing_dataset/02.jpg 
Facial landmarks detected in testing_dataset/02.jpg
Face detected in testing_dataset/03.jpg 
Facial landmarks detected in testing_dataset/03.jpg
Face detected in testing_dataset/04.jpg 
Facial landmarks detected in testing_dataset/04.jpg
Face detected in testing_dataset/05.jpg 
Facial landmarks detected in testing_dataset/05.jpg
Face detected in testing_dataset/06.jpg 
Facial landmarks detected in testing_dataset/06.jpg
Face detected in testing_dataset/07.jpg 
Facial landmarks detected in testing_dataset/07.jpg
Face detected in testing_dataset/08.jpg 
Facial landmarks detected in testing_dataset/08.jpg
Face detected in testing_dataset/09.jpg 
Facial landmarks detected in testing_dataset/09.jpg
Face detected in testing_dataset/10.jpg 
Facial landmarks detected in testing_dataset/10.jpg
Face detected in testing_dataset/11.jpg 
Facial landmarks detected in 

In [5]:
import cv2
import dlib
from imutils import face_utils
import numpy as np
from scipy.spatial import distance as dist
import os

def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

def test_face_and_landmark_detection(image_path, predictor):
    haar_detector = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    dlib_detector = dlib.get_frontal_face_detector()
    
    image = cv2.imread(image_path)
    if image is None:
        print(f"Image file {image_path} cannot be loaded.")
        return
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    haar_faces = haar_detector.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    dlib_faces = dlib_detector(gray)

    if len(haar_faces) > 0 or len(dlib_faces) > 0:
        if len(haar_faces) > 0:
            for (x, y, w, h) in haar_faces:
                rect = dlib.rectangle(int(x), int(y), int(x + w), int(y + h))
                try:
                    shape = predictor(gray, rect)
                    shape = face_utils.shape_to_np(shape)
                    
                    if len(shape) == 68:
                        left_eye = shape[36:42]
                        right_eye = shape[42:48]
                        
                        left_ear = eye_aspect_ratio(left_eye)
                        right_ear = eye_aspect_ratio(right_eye)
                        
                        print(f"{image_path}: Left EAR: {left_ear:.2f}, Right EAR: {right_ear:.2f}")
                except Exception as e:
                    print(f"Error with Haar Cascade landmarks detection: {e}")
                    
        if len(dlib_faces) > 0:
            for rect in dlib_faces:
                try:
                    shape = predictor(gray, rect)
                    shape = face_utils.shape_to_np(shape)
                    
                    if len(shape) == 68:
                        left_eye = shape[36:42]
                        right_eye = shape[42:48]
                        
                        left_ear = eye_aspect_ratio(left_eye)
                        right_ear = eye_aspect_ratio(right_eye)
                        
                        print(f"{image_path}: Left EAR: {left_ear:.2f}, Right EAR: {right_ear:.2f}")
                except Exception as e:
                    print(f"Error with Dlib landmarks detection: {e}")
    else:
        print(f"No face detected in {image_path} using both methods.")

predictor_path = 'shape_predictor_68_face_landmarks.dat'
predictor = dlib.shape_predictor(predictor_path)

for i in range(1, 18):
    image_path = f"testing_dataset/{i:02d}.jpg"
    if not os.path.exists(image_path):
        print(f"Image file {image_path} does not exist.")
        continue

    test_face_and_landmark_detection(image_path, predictor)


testing_dataset/01.jpg: Left EAR: 0.27, Right EAR: 0.19
testing_dataset/01.jpg: Left EAR: 0.35, Right EAR: 0.23
testing_dataset/02.jpg: Left EAR: 0.30, Right EAR: 0.39
testing_dataset/03.jpg: Left EAR: 0.24, Right EAR: 0.24
testing_dataset/04.jpg: Left EAR: 0.23, Right EAR: 0.26
testing_dataset/05.jpg: Left EAR: 0.35, Right EAR: 0.28
testing_dataset/06.jpg: Left EAR: 0.37, Right EAR: 0.31
testing_dataset/06.jpg: Left EAR: 0.30, Right EAR: 0.29
testing_dataset/07.jpg: Left EAR: 0.28, Right EAR: 0.27
testing_dataset/07.jpg: Left EAR: 0.30, Right EAR: 0.30
testing_dataset/08.jpg: Left EAR: 0.26, Right EAR: 0.26
testing_dataset/08.jpg: Left EAR: 0.23, Right EAR: 0.26
testing_dataset/09.jpg: Left EAR: 0.30, Right EAR: 0.25
testing_dataset/09.jpg: Left EAR: 0.31, Right EAR: 0.27
testing_dataset/10.jpg: Left EAR: 0.32, Right EAR: 0.31
testing_dataset/10.jpg: Left EAR: 0.35, Right EAR: 0.38
testing_dataset/11.jpg: Left EAR: 0.23, Right EAR: 0.25
testing_dataset/11.jpg: Left EAR: 0.21, Right EA

In [6]:
import cv2
import numpy as np
import dlib
from imutils import face_utils
import os

def lip_distance(shape):
    top_lip = shape[50:53]
    top_lip = np.concatenate((top_lip, shape[61:64]))
    low_lip = shape[56:59]
    low_lip = np.concatenate((low_lip, shape[65:68]))
    top_mean = np.mean(top_lip, axis=0)
    low_mean = np.mean(low_lip, axis=0)
    distance = abs(top_mean[1] - low_mean[1])
    return distance

def test_face_and_landmark_detection(image_path, predictor):
    haar_detector = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    dlib_detector = dlib.get_frontal_face_detector()
    
    image = cv2.imread(image_path)
    if image is None:
        print(f"Image file {image_path} cannot be loaded.")
        return
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    haar_faces = haar_detector.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    dlib_faces = dlib_detector(gray)

    if len(haar_faces) > 0 or len(dlib_faces) > 0:
        if len(haar_faces) > 0:
            for (x, y, w, h) in haar_faces:
                rect = dlib.rectangle(int(x), int(y), int(x + w), int(y + h))
                try:
                    shape = predictor(gray, rect)
                    shape = face_utils.shape_to_np(shape)
                    
                    if len(shape) == 68:
                        distance = lip_distance(shape)
                        print(f"{image_path}: Lip Distance: {distance:.2f}")
                except Exception as e:
                    print(f"Error with Haar Cascade landmarks detection: {e}")
                    
        if len(dlib_faces) > 0:
            for rect in dlib_faces:
                try:
                    shape = predictor(gray, rect)
                    shape = face_utils.shape_to_np(shape)
                    
                    if len(shape) == 68:
                        distance = lip_distance(shape)
                        print(f"{image_path}: Lip Distance: {distance:.2f}")
                except Exception as e:
                    print(f"Error with Dlib landmarks detection: {e}")
    else:
        print(f"No face detected in {image_path} using both methods.")

predictor_path = 'shape_predictor_68_face_landmarks.dat'
predictor = dlib.shape_predictor(predictor_path)

for i in range(1, 18):
    image_path = f"testing_dataset/{i:02d}.jpg"
    if not os.path.exists(image_path):
        print(f"Image file {image_path} does not exist.")
        continue

    test_face_and_landmark_detection(image_path, predictor)


testing_dataset/01.jpg: Lip Distance: 11.83
testing_dataset/01.jpg: Lip Distance: 13.33
testing_dataset/02.jpg: Lip Distance: 7.83
testing_dataset/03.jpg: Lip Distance: 19.50
testing_dataset/04.jpg: Lip Distance: 20.00
testing_dataset/05.jpg: Lip Distance: 21.00
testing_dataset/06.jpg: Lip Distance: 13.50
testing_dataset/06.jpg: Lip Distance: 16.50
testing_dataset/07.jpg: Lip Distance: 16.50
testing_dataset/07.jpg: Lip Distance: 25.50
testing_dataset/08.jpg: Lip Distance: 12.33
testing_dataset/08.jpg: Lip Distance: 9.33
testing_dataset/09.jpg: Lip Distance: 7.50
testing_dataset/09.jpg: Lip Distance: 9.00
testing_dataset/10.jpg: Lip Distance: 10.83
testing_dataset/10.jpg: Lip Distance: 3.67
testing_dataset/11.jpg: Lip Distance: 19.17
testing_dataset/11.jpg: Lip Distance: 20.33
testing_dataset/12.jpg: Lip Distance: 14.17
testing_dataset/13.jpg: Lip Distance: 3.33
testing_dataset/13.jpg: Lip Distance: 3.17
testing_dataset/14.jpg: Lip Distance: 11.00
testing_dataset/15.jpg: Lip Distance: 1

In [7]:
import cv2
import numpy as np
import dlib
from imutils import face_utils
import os

def lip_distance(shape):
    top_lip = shape[50:53]
    top_lip = np.concatenate((top_lip, shape[61:64]))
    low_lip = shape[56:59]
    low_lip = np.concatenate((low_lip, shape[65:68]))
    top_mean = np.mean(top_lip, axis=0)
    low_mean = np.mean(low_lip, axis=0)
    distance = abs(top_mean[1] - low_mean[1])
    return distance

def visualize_lip_distance(image, shape, distance):
    top_lip = shape[50:53]
    top_lip = np.concatenate((top_lip, shape[61:64]))
    low_lip = shape[56:59]
    low_lip = np.concatenate((low_lip, shape[65:68]))
    
    for (x, y) in top_lip:
        cv2.circle(image, (int(x), int(y)), 2, (0, 255, 0), -1)
    for (x, y) in low_lip:
        cv2.circle(image, (int(x), int(y)), 2, (0, 0, 255), -1)
    
    top_mean = np.mean(top_lip, axis=0)
    low_mean = np.mean(low_lip, axis=0)
    cv2.line(image, (int(top_mean[0]), int(top_mean[1])), (int(low_mean[0]), int(low_mean[1])), (255, 0, 0), 2)
    cv2.putText(image, f"Lip Distance: {distance:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

def test_face_and_landmark_detection(image_path, predictor):
    haar_detector = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    dlib_detector = dlib.get_frontal_face_detector()
    
    image = cv2.imread(image_path)
    if image is None:
        print(f"Image file {image_path} cannot be loaded.")
        return
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    haar_faces = haar_detector.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    dlib_faces = dlib_detector(gray)

    if len(haar_faces) > 0 or len(dlib_faces) > 0:
        if len(haar_faces) > 0:
            for (x, y, w, h) in haar_faces:
                rect = dlib.rectangle(int(x), int(y), int(x + w), int(y + h))
                try:
                    shape = predictor(gray, rect)
                    shape = face_utils.shape_to_np(shape)
                    
                    if len(shape) == 68:
                        distance = lip_distance(shape)
                        visualize_lip_distance(image, shape, distance)
                        cv2.imshow("Lip Distance Visualization", image)
                        cv2.waitKey(0)
                        cv2.destroyAllWindows()
                except Exception as e:
                    print(f"Error with Haar Cascade landmarks detection: {e}")
                    
        if len(dlib_faces) > 0:
            for rect in dlib_faces:
                try:
                    shape = predictor(gray, rect)
                    shape = face_utils.shape_to_np(shape)
                    
                    if len(shape) == 68:
                        distance = lip_distance(shape)
                        visualize_lip_distance(image, shape, distance)
                        cv2.imshow("Lip Distance Visualization", image)
                        cv2.waitKey(0)
                        cv2.destroyAllWindows()
                except Exception as e:
                    print(f"Error with Dlib landmarks detection: {e}")
    else:
        print(f"No face detected in {image_path} using both methods.")

predictor_path = 'shape_predictor_68_face_landmarks.dat'
predictor = dlib.shape_predictor(predictor_path)

for i in range(1, 18):
    image_path = f"testing_dataset/{i:02d}.jpg"
    if not os.path.exists(image_path):
        print(f"Image file {image_path} does not exist.")
        continue

    test_face_and_landmark_detection(image_path, predictor)
